# Local Controller Noise Analysis (Take 2)

Required penalty energy $E_p$ vs RMS noise amplitude for selected correlation times $\tau_X$.

This notebook follows the exact method from `reference/3qubit_real_numbers_1f_nose.ipynb`:
- Uses `sesolve` (closed system) with OU noise on control fields
- Single realization per point (fixed seed for reproducibility)
- Sinusoidal RAP pulses
- Fidelity = projection onto |1_L⟩

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from qutip import sesolve
from scipy.interpolate import interp1d

sys.path.insert(0, os.path.dirname(os.getcwd()))
from qec_config import (
    QECConfig, BaconShorConfig, PlotConfig,
    sigma_from_rms, ou_trace
)

PlotConfig.apply()

# Output directories
OUTPUT_DIR = Path('../figs/control_noise')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = Path('../data/control_noise')
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output: {OUTPUT_DIR}")
print(f"Data: {DATA_DIR}")

In [ ]:
# === Platform Parameters (IBM - matching reference notebook) ===
OMEGA_MAX = 2 * np.pi * 12.5e6  # 12.5 MHz Rabi frequency
T_MAX = 4e-6                     # 4 us protocol duration

# Initialize configurations with reference parameters
rep = QECConfig(omega_max=OMEGA_MAX, T_max=T_MAX, n_points=51)
bs = BaconShorConfig(omega_max=OMEGA_MAX, T_max=T_MAX, n_points=51)

# Target fidelity
TARGET_FID = 0.99

# Ep grid (rad/s)
Ep_grid = np.linspace(1e4, 2 * np.pi * 150e6, 100)

# RMS amplitude sweep (as fraction of omega_max)
rms_X_fracs = np.linspace(0.1, 1.0, 10)  # 10% to 100%
rms_X_grid = rms_X_fracs * OMEGA_MAX  # rad/s

# Correlation time grid
tau_X_grid = np.linspace(0.1 * T_MAX, 1.0 * T_MAX, 250)

# Print info
rep.info()
print(f"\nSweep Parameters:")
print(f"  Ep_grid: {Ep_grid[0]/(2*np.pi*1e6):.2f} - {Ep_grid[-1]/(2*np.pi*1e6):.1f} MHz ({len(Ep_grid)} points)")
print(f"  RMS range: {rms_X_fracs[0]*100:.0f}% - {rms_X_fracs[-1]*100:.0f}% ({len(rms_X_fracs)} points)")
print(f"  tau_X range: {tau_X_grid[0]*1e6:.2f} - {tau_X_grid[-1]*1e6:.2f} us ({len(tau_X_grid)} points)")
print(f"  Target fidelity: {TARGET_FID}")

In [ ]:
def Ep_for_target_fid(config, sigma_X_val, tau_X_val, Ep_grid, target_fid):
    """
    Find minimum Ep that achieves target fidelity under OU control noise.
    
    Uses sesolve (closed system) with single noise realization (fixed seed).
    
    Parameters
    ----------
    config : QECConfig or BaconShorConfig
        QEC configuration object
    sigma_X_val : float
        OU noise strength (rad/s)
    tau_X_val : float
        OU correlation time (s)
    Ep_grid : array
        Grid of Ep values to search (rad/s)
    target_fid : float
        Target fidelity threshold
    
    Returns
    -------
    float
        Minimum Ep achieving target fidelity, or np.nan if not achieved
    """
    tlist = config.t_list
    n_qubits = 4 if config.code_name == 'bacon_shor' else 3
    
    # Get operators from config
    X_L = config.X_L
    Z_L = config.Z_L
    S_penalty = config.get_stabilizer_penalty()
    X_ops = config.get_single_qubit_X_ops()
    
    # Initial state and projector
    psi0 = config.logical_zero
    proj_1L = config.logical_one * config.logical_one.dag()
    
    # Pulse functions (sinusoidal)
    omega_max = config.omega_max
    T_max = config.T_max
    
    def omega_t(t):
        return omega_max * np.sin(np.pi * t / T_max)
    
    def delta_t(t):
        return -omega_max * np.cos(np.pi * t / T_max)
    
    for Ep_val in Ep_grid:
        # Fixed seed for reproducibility (matching reference)
        rng = np.random.default_rng(42)
        
        # Generate noise traces and create interpolation functions
        noise_funcs = []
        for _ in range(n_qubits):
            trace = ou_trace(tlist, tau_X_val, sigma_X_val, rng)
            interp = interp1d(tlist, trace, kind='linear', fill_value='extrapolate')
            noise_funcs.append(interp)
        
        # Build Hamiltonian with noise as callable coefficients
        # Note: args=None default required for QuTiP coefficient testing
        H = [
            [X_L, lambda t, args=None: omega_t(t)],
            [Z_L, lambda t, args=None: delta_t(t)],
            -Ep_val * S_penalty,
        ]
        
        # Add noise terms - need to capture the function in closure properly
        for i in range(n_qubits):
            nf = noise_funcs[i]
            H.append([X_ops[i], lambda t, args=None, f=nf: float(f(t))])
        
        # Solve
        sol = sesolve(H, psi0, tlist, e_ops=[proj_1L])
        final_fid = np.real(sol.expect[0][-1])
        
        if final_fid >= target_fid:
            return Ep_val
    
    return np.nan

print("Fidelity function defined (uses config operators)")

In [ ]:
# === Compute Ep map for Repetition Code ===
print("Computing Ep map for Repetition Code [[3,1,3]]...")
print("="*60)

Ep_map_rep = np.zeros((len(rms_X_grid), len(tau_X_grid)))

for j, tx in enumerate(tqdm(tau_X_grid, desc="tau_X sweep")):
    for i, rmsX in enumerate(rms_X_grid):
        sigmaX = sigma_from_rms(rmsX, tx)
        Ep_map_rep[i, j] = Ep_for_target_fid(rep, sigmaX, tx, Ep_grid, TARGET_FID)

print("="*60)
print("Done!")

In [ ]:
# === Compute Ep map for Bacon-Shor Code ===
print("Computing Ep map for Bacon-Shor [[4,1,1,2]]...")
print("="*60)

Ep_map_bs = np.zeros((len(rms_X_grid), len(tau_X_grid)))

for j, tx in enumerate(tqdm(tau_X_grid, desc="tau_X sweep")):
    for i, rmsX in enumerate(rms_X_grid):
        sigmaX = sigma_from_rms(rmsX, tx)
        Ep_map_bs[i, j] = Ep_for_target_fid(bs, sigmaX, tx, Ep_grid, TARGET_FID)

print("="*60)
print("Done!")

In [ ]:
# === Save computed data ===
data = {
    # Grids
    'rms_X_fracs': rms_X_fracs,
    'rms_X_grid': rms_X_grid,
    'tau_X_grid': tau_X_grid,
    'Ep_grid': Ep_grid,
    # Results
    'Ep_map_rep': Ep_map_rep,
    'Ep_map_bs': Ep_map_bs,
    # Parameters
    'omega_max': OMEGA_MAX,
    'T_max': T_MAX,
    'target_fid': TARGET_FID,
}

np.savez_compressed(DATA_DIR / f'Ep_heatmap_RMS_vs_tau_X_F{TARGET_FID}_data.npz', **data)
print(f"Saved data to {DATA_DIR / f'Ep_heatmap_RMS_vs_tau_X_F{TARGET_FID}_data.npz'}")

In [ ]:
# === 1D Plot: Ep vs RMS Amplitude for selected tau_X values ===

# Select tau_X values to plot
n_tau_samples = 6
tau_indices = np.linspace(0, len(tau_X_grid)-1, n_tau_samples, dtype=int)

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Color palette
colors = plt.cm.viridis(np.linspace(0.15, 0.85, n_tau_samples))

# Plot Repetition Code
for i, tau_idx in enumerate(tau_indices):
    tau_val = tau_X_grid[tau_idx]
    Ep_slice = Ep_map_rep[:, tau_idx]
    
    ax1.plot(rms_X_grid / (2*np.pi*1e6), Ep_slice / (2*np.pi*1e6),
             'o-', color=colors[i], linewidth=2, markersize=6,
             label=rf'$\tau_X$ = {tau_val*1e6:.2f} $\mu$s')

ax1.set_ylabel(r'Required $E_p$ [MHz]', fontsize=16)
ax1.set_title(r'(a) Repetition $[[3,1,3]]$', fontsize=18)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=14)
ax1.legend(fontsize=10, loc='upper left')

# Plot Bacon-Shor Code
for i, tau_idx in enumerate(tau_indices):
    tau_val = tau_X_grid[tau_idx]
    Ep_slice = Ep_map_bs[:, tau_idx]
    
    ax2.plot(rms_X_grid / (2*np.pi*1e6), Ep_slice / (2*np.pi*1e6),
             's-', color=colors[i], linewidth=2, markersize=6,
             label=rf'$\tau_X$ = {tau_val*1e6:.2f} $\mu$s')

ax2.set_title(r'(b) Bacon-Shor $[[4,1,1,2]]$', fontsize=18)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=14)
ax2.legend(fontsize=10, loc='upper left')

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)

# Shared x label
fig.text(0.5, 0.02, r'RMS Amplitude [MHz]', ha='center', fontsize=16)

plt.show()
print("Done: 1D plot of Ep vs RMS Amplitude for selected tau_X values.")

In [ ]:
# === Save figures ===
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.pdf', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.svg', bbox_inches='tight')
fig.savefig(OUTPUT_DIR / f'Ep_vs_RMS_comparison_F{TARGET_FID}.png', dpi=300, bbox_inches='tight')

print(f"Saved figures to {OUTPUT_DIR}")
for f in sorted(OUTPUT_DIR.glob(f'Ep_vs_RMS*F{TARGET_FID}*')):
    print(f"  {f.name}")

In [ ]:
# === Summary ===
print("\n" + "="*70)
print(f"SUMMARY: Required Ep [MHz] for F >= {TARGET_FID}")
print("="*70)

# Select a few tau values for display
tau_display_indices = [0, len(tau_X_grid)//4, len(tau_X_grid)//2, -1]

print(f"\nRepetition Code [[3,1,3]]:")
print(f"{'RMS [MHz]':<12}", end="")
for idx in tau_display_indices:
    tau_us = tau_X_grid[idx] * 1e6
    print(f"  tau={tau_us:.2f}us", end="")
print()
print("-"*60)
for i, rmsX in enumerate(rms_X_grid):
    rms_MHz = rmsX / (2*np.pi*1e6)
    print(f"{rms_MHz:8.2f}    ", end="")
    for idx in tau_display_indices:
        Ep_MHz = Ep_map_rep[i, idx] / (2*np.pi*1e6)
        if np.isnan(Ep_MHz):
            print(f"      >150", end="")
        else:
            print(f"    {Ep_MHz:6.1f}", end="")
    print()

print(f"\nBacon-Shor Code [[4,1,1,2]]:")
print(f"{'RMS [MHz]':<12}", end="")
for idx in tau_display_indices:
    tau_us = tau_X_grid[idx] * 1e6
    print(f"  tau={tau_us:.2f}us", end="")
print()
print("-"*60)
for i, rmsX in enumerate(rms_X_grid):
    rms_MHz = rmsX / (2*np.pi*1e6)
    print(f"{rms_MHz:8.2f}    ", end="")
    for idx in tau_display_indices:
        Ep_MHz = Ep_map_bs[i, idx] / (2*np.pi*1e6)
        if np.isnan(Ep_MHz):
            print(f"      >150", end="")
        else:
            print(f"    {Ep_MHz:6.1f}", end="")
    print()

print("="*70)